In [ ]:
import pandas as pd
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objects as go
from plotly.io import write_image
import pickle
import alphashape

In [ ]:
df = pd.read_parquet('semantics.parquet')
df["cluster"] = df["cluster"].astype('category')

In [ ]:
# Plot valence, humanity, and physicality
df['annotation'] = df[['territory_name', 'cluster_topic', 'sfw_term','sfw_definition']].apply(lambda row: f'{row.territory_name} ({row.cluster_topic}) | {row.sfw_term}', axis=1)

fig = px.scatter(df[df.cluster!=-1], x='umap_1', y='umap_2',
                 color='valence',
                 hover_name='annotation',
                 hover_data={'umap_1': False, 'umap_2': False},
                 title='Emotional Valence')
fig.update_traces(marker={'size': 2})
fig.update_layout(width=800,
                  height=500,
                  plot_bgcolor='white',
                  showlegend=False,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.show()
write_image(fig, 'emotional valence.png')

fig = px.scatter(df[df.cluster!=-1], x='umap_1', y='umap_2',
                 color='physicality',
                 hover_name='annotation',
                 hover_data={'umap_1': False, 'umap_2': False},
                 title='Physicality')
fig.update_traces(marker={'size': 2})
fig.update_layout(width=800,
                  height=500,
                  plot_bgcolor='white',
                  showlegend=False,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.show()
write_image(fig, 'physicality.png')

fig = px.scatter(df[df.cluster!=-1], x='umap_1', y='umap_2',
                 color='humanity',
                 hover_name='annotation',
                 hover_data={'umap_1': False, 'umap_2': False},
                 title='Humanity')
fig.update_traces(marker={'size': 2})
fig.update_layout(width=800,
                  height=500,
                  plot_bgcolor='white',
                  showlegend=False,
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.show()
write_image(fig, 'humanity.png')

In [ ]:
# Compute the enclosing hulls for each island

# Initialize dictionary to store hull points for each cluster
land_polygons_dict = {}

# Compute hull points for each cluster and store in dictionary
for cluster_id, group in df.groupby('cluster', observed=True):
    if group['cluster'].iloc[0].astype(int)>=0:
        alpha = 25 # The highest alpha to try (most reticulated coastline)
        got_hull = False
        while not got_hull:
            # Keep trying consecutively lower alphas until we get a polygon that works
            try:
                points = group[['umap_1', 'umap_2']]
                hull = alphashape.alphashape(points, alpha=alpha)  # Adjust alpha for concavity
                if hull.geom_type == 'Polygon':
                    hull_points = list(zip(*hull.exterior.coords.xy))
                elif hull.geom_type == 'MultiPolygon':  # Handle multiple polygons if needed
                    hull_points = [list(zip(*poly.exterior.coords.xy)) for poly in hull]
                    print('multi')
                else:
                    hull_points = []
                land_polygons_dict[cluster_id] = hull_points
                got_hull = True
            except:
                alpha = alpha - 1

In [ ]:
# Map the valence, humanity, and physicality scores to colors

def color_name_to_rgb(name):
    if name == 'orange':
            return [220, 185, 0]
    if name == 'green':
            return [0, 185, 0]
    if name == 'grey':
            return [210, 210, 210]

def map_value_to_color(value, low_color='orange', high_color='green', value_min=1, value_max=5, saturation=1):
    low_color = color_name_to_rgb(low_color)
    high_color = color_name_to_rgb(high_color)

    # Normalize
    normalized_value = (value - value_min) / (value_max - value_min)
    
    # Interpolate
    interpolated_color = [low_color[i] + (high_color[i] - low_color[i]) * normalized_value for i in range(3)]
    
    # Desaturate
    interpolated_color = [int(v*saturation) for v in interpolated_color]

    return '#{:02x}{:02x}{:02x}'.format(*interpolated_color)

def mix_hex_colors(hex1, hex2):
    # Convert hex to RGB
    rgb1 = int(hex1[1:], 16)
    rgb2 = int(hex2[1:], 16)
    
    # Extract RGB components
    r1, g1, b1 = (rgb1 >> 16) & 0xFF, (rgb1 >> 8) & 0xFF, rgb1 & 0xFF
    r2, g2, b2 = (rgb2 >> 16) & 0xFF, (rgb2 >> 8) & 0xFF, rgb2 & 0xFF
    
    # Average the RGB components
    r_avg = (r1 + r2) // 2
    g_avg = (g1 + g2) // 2
    b_avg = (b1 + b2) // 2
    
    # Convert back to hex
    return f'#{r_avg:02x}{g_avg:02x}{b_avg:02x}'

# Placeholder for your hex color function
def hpv_to_color(h, p, v, saturation=1):
    v = (v-1)/3.5 # rescale v to 0 to 1 range
    saturation = saturation * (v+0.75)/1.75
    c1 = map_value_to_color(h, saturation=saturation)
    c2 = map_value_to_color(p, low_color='grey', saturation=saturation)
    
    return mix_hex_colors(c1, c2)
    # return f'#{h:02x}{p:02x}{(h+p):02x}'

# Test
humanity = [1, 3, 5]
physicality = [1, 3, 5]

# High valence
colors = [hpv_to_color(h, p, 5) for h in humanity for p in physicality]
x_values = [h for h in humanity for _ in physicality]
y_values = [p for _ in humanity for p in physicality]

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=x_values, 
    y=y_values, 
    mode='markers',
    marker=dict(color=colors, size=60, symbol='square')
))

# Adjust layout
fig.update_layout(width=400,
                  height=400,
                  title='High valence',
                  xaxis_title='Humanity',
                  yaxis_title='Physicality',
                  xaxis=dict(tickvals=humanity, scaleanchor="y", scaleratio=1),
                  yaxis=dict(tickvals=physicality),
                  plot_bgcolor='white')

# Show plot
fig.show()
write_image(fig, 'land_colors_high_valence.png')

# Low valence
colors = [hpv_to_color(h, p, 1) for h in humanity for p in physicality]
x_values = [h for h in humanity for _ in physicality]
y_values = [p for _ in humanity for p in physicality]

# Create the plot
fig = go.Figure(data=go.Scatter(
    x=x_values, 
    y=y_values, 
    mode='markers',
    marker=dict(color=colors, size=60, symbol='square')
))

# Adjust layout
fig.update_layout(width=400,
                  height=400,
                  title='Low valence',
                  xaxis_title='Humanity',
                  yaxis_title='Physicality',
                  xaxis=dict(tickvals=humanity, scaleanchor="y", scaleratio=1),
                  yaxis=dict(tickvals=physicality),
                  plot_bgcolor='white')

# Show plot
fig.show()
write_image(fig, 'land_colors_low_valence.png')

In [ ]:
# Plot and save to a .html

df['annotation'] = df[['territory_name', 'cluster_topic', 'sfw_term','sfw_definition']].apply(lambda row: f'{row.territory_name} ({row.cluster_topic})<br>{row.sfw_term} - {row.sfw_definition}', axis=1)

def cluster_to_color(cluster, saturation=1):
    row = df[df.cluster==cluster].iloc[0]
    if row.cluster.astype(int)<0:
        return 'rgb(230, 230, 255)'
    humanity = row.humanity
    physicality = row.physicality
    valence = row.valence
    return hpv_to_color(humanity, physicality, valence, saturation)#map_value_to_color(humanity, saturation=saturation)

color_map = {cluster: cluster_to_color(cluster, 0.9) for cluster in df.cluster.unique()}

# marked_cluster = df[df.term == 'fire']['cluster'].to_list()
# for i in marked_cluster:
#     if i!=-1:
#         color_map[i] = 'red'

import plotly.graph_objects as go
fig = go.Figure()

for cluster in land_polygons_dict:
    polygon = land_polygons_dict[cluster]
    # fig.add_trace(go.Scatter(x=[p[0] for p in polygon], y=[p[1] for p in polygon], mode='lines', fill='toself'))
    color = cluster_to_color(cluster)
    fig.add_trace(go.Scatter(
                            x=[v[0] for v in polygon], 
                            y=[v[1] for v in polygon], 
                            mode='lines', 
                            fill='toself',
                            fillcolor=color, # fill color
                            line=dict(color=color)  # outline color
                        ))
    
scatter_plot = px.scatter(df, x='umap_1', y='umap_2',
                                color='cluster',
                                hover_name='annotation',
                                hover_data={'umap_1': False, 'umap_2': False, 'cluster': False},
                                title='UMAP Projection',
                                color_discrete_map=color_map)
scatter_plot.update_traces(marker={'size': 2.5})

for trace in scatter_plot.data:
    fig.add_trace(trace)

fig.update_layout(title={'text': '<b>Lexiconia</b><br><a href="https://github.com/jjdhunt/lexiconia">https://github.com/jjdhunt/lexiconia</a>',
                         'y':0.95,
                         'x':0.5,
                         'xanchor': 'center',
                         'yanchor': 'top'},
                  autosize=True,
                  plot_bgcolor='skyblue',
                  showlegend=False,
                  xaxis=dict(showgrid=False,
                             showticklabels=False,
                             zeroline=False,
                             scaleanchor='y',  # This makes x-axis scale depend on y-axis
                             scaleratio=1),  # This ensures the scale ratio is 1:1),
                  yaxis=dict(showgrid=False,
                             showticklabels=False,
                             zeroline=False),
                  dragmode='pan')


config = {
    'scrollZoom': True,
    'responsive': True,  # Make plot responsive to window size
    'modeBarButtonsToRemove': [
        'zoom2d', 'pan2d', 'select2d', 'lasso2d', 'zoomIn2d', 'zoomOut2d', 
        'autoScale2d', 'resetScale2d', 'hoverClosestCartesian', 
        'hoverCompareCartesian', 'toggleSpikelines'
    ]
}

# fig.show()
pyo.plot(fig, filename='lexiconia.html', config=config, auto_open=True)